In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


# Se levanta el split hecho en el NoteBook anterior

In [2]:
X_train = pd.read_csv("X_train")
X_train.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,anio_publ
0,0.0,3.0,2.0,3.0,196.0,160.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2014
1,20.0,4.0,2.0,2.0,245.0,196.0,104734.0,NaN,NaN,0.0,0.0,0.0,1.0,0.0,2012
2,5.0,3.0,1.0,2.0,108.0,120.0,58952.0,NaN,NaN,0.0,0.0,0.0,1.0,1.0,2015
3,10.0,3.0,1.0,3.0,200.0,147.0,49066.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2016
4,NaN,NaN,0.0,NaN,NaN,250.0,24892.0,19.23339,-99.166991,0.0,0.0,0.0,0.0,0.0,2016


In [3]:
X_test = pd.read_csv("X_test")

In [4]:
y_train = pd.read_csv("y_train")

In [5]:
y_test = pd.read_csv("y_test")

# Se levantan todos los XGBoost guardados

In [6]:
xgb_created = []

## Atencion! n tiene que coincidir con la cantidad guardada!

In [7]:
n = 100

In [8]:
for i in range(n):
    current_xgb = xgb.XGBRegressor()
    current_xgb.load_model("xgb_{}".format(i))
    xgb_created.append(current_xgb)

# Se arma el DF de precios para entrenar el RF

In [9]:
XGB_ens_train = pd.DataFrame(index = X_train.index.copy())

In [10]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_train)
    XGB_ens_train['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
XGB_ens_train.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,2.220148e+06,2049952.625,2120484.750,2148481.500,2172930.250,2074102.75,2134281.75,2070373.500,2095271.125,2175037.500,...,2138135.50,2.019896e+06,2.144939e+06,2280565.000,2.158493e+06,2098802.000,2157095.750,2026265.375,2.091589e+06,2.229746e+06
1,1.836807e+06,1859878.250,1844559.625,1713691.250,1905118.750,1764503.50,1694591.25,1893943.000,1779136.500,1872670.625,...,1889341.25,1.766836e+06,1.808580e+06,1885901.250,1.809297e+06,1825860.125,1808524.750,1800260.000,1.711154e+06,1.856461e+06
2,9.938113e+05,1062887.250,1051915.750,1089754.750,1079227.125,1087252.75,1048740.50,1052974.750,1092731.000,993707.625,...,1035377.50,9.965896e+05,1.020327e+06,1028185.250,9.920494e+05,1069735.125,1056923.375,980369.125,9.816222e+05,9.775687e+05
3,2.087953e+06,1899654.250,1940235.625,2107745.500,2016174.000,2029273.75,1863063.75,2006435.125,2063431.500,2011218.375,...,2116243.50,1.941300e+06,1.804553e+06,1943244.125,1.831882e+06,1988650.625,2204776.000,2101752.000,2.133853e+06,1.748996e+06
4,2.353710e+06,2493387.500,2447289.250,2024587.625,2543587.000,3000341.75,2756347.75,3080066.000,3049393.250,2406472.000,...,2433627.75,2.532436e+06,2.643687e+06,1885165.500,2.439610e+06,2531846.500,2113644.750,2153232.000,2.858623e+06,2.577500e+06


In [11]:
len(XGB_ens_train)

168000

In [12]:
len(y_train)

168000

## Exporto el XGB_ens_train para poder buscar hiperparametros

In [13]:
XGB_ens_train.to_csv("XGB_ens_train", index = False)

# Entreno el XGB con XGB_ens_train

In [14]:
model_XGB = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1.0, gamma=2,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=40, min_child_weight=10, missing=None, n_estimators=200,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1.0, verbosity=1)
model_XGB.fit(XGB_ens_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1.0, gamma=2,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=40, min_child_weight=10, missing=None, n_estimators=200,
       n_jobs=-1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1.0, verbosity=1)

# Testeo el modelo final

## Armo el DF para X_test

In [15]:
XGB_ens_test = pd.DataFrame(index = X_test.index.copy())


In [16]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test)
    XGB_ens_test['pred_{}'.format(i)] = current_price_pred
XGB_ens_test.head()

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1.340262e+06,1201193.875,1384029.875,1.302512e+06,1.284428e+06,1.289528e+06,1.292242e+06,1.346360e+06,1.333442e+06,1359851.625,...,1.285571e+06,1.247251e+06,1.208817e+06,1.236398e+06,1211372.875,1.251900e+06,1.289309e+06,1.350972e+06,1.253407e+06,1.240814e+06
1,2.688287e+06,2682824.250,2415446.250,2.371418e+06,2.403966e+06,2.522047e+06,2.336102e+06,2.455772e+06,2.518206e+06,2436715.750,...,2.452529e+06,2.588680e+06,2.542316e+06,2.401415e+06,2621578.250,2.783493e+06,2.464749e+06,2.698421e+06,2.552196e+06,2.545391e+06
2,2.747507e+06,2939026.500,2789936.000,2.683204e+06,3.167737e+06,2.800803e+06,2.888054e+06,2.510845e+06,3.089269e+06,2813100.500,...,2.414746e+06,2.305971e+06,2.121726e+06,3.408145e+06,2935359.000,2.906436e+06,2.706738e+06,2.598074e+06,2.799992e+06,2.861804e+06
3,1.415355e+06,1535345.250,1448461.625,1.305252e+06,1.362654e+06,1.651582e+06,1.328473e+06,1.349953e+06,1.378403e+06,1365628.750,...,1.603374e+06,1.540706e+06,1.520938e+06,1.502945e+06,1311053.375,1.528065e+06,1.505249e+06,1.339692e+06,1.516940e+06,1.367100e+06
4,3.761519e+05,366148.375,214707.375,3.194135e+05,3.577163e+05,2.426129e+05,2.637887e+05,2.963697e+05,2.841103e+05,356606.750,...,2.903783e+05,3.263117e+05,2.657470e+05,3.757636e+05,424055.375,3.286349e+05,3.396963e+05,3.536586e+05,3.017619e+05,3.967939e+05


## Se hacen las dos predicciones necesarias

In [17]:
pred_train = model_XGB.predict(XGB_ens_train)
pred_test = model_XGB.predict(XGB_ens_test)

### Metrica de Mean Absolute Error (La de Kaggle) ?? (Hay que preguntar)

In [18]:
mae_train = np.sqrt(sklearn.metrics.mean_absolute_error(y_train, pred_train))
mae_test = np.sqrt(sklearn.metrics.mean_absolute_error(y_test, pred_test))
#print(f"MAE train: {mea_train:.5f}")
#print(f"MAE test: {mea_test:.5f}")
print("MAE train: "+str(mae_train))
print("MAE test: "+str(mae_test))

MAE train: 269.48637217206056
MAE test: 797.4865470276285


#### Overfittea una bocha! Pero eso quiere decir que hay mucho rango para mejorar :)

In [19]:
model_XGB.feature_importances_

array([0.00171331, 0.02461813, 0.0016746 , 0.00151684, 0.00162529,
       0.00146542, 0.003168  , 0.00181654, 0.00156914, 0.00167924,
       0.00149625, 0.00312007, 0.0016863 , 0.00192684, 0.00155901,
       0.00162861, 0.00184474, 0.00180627, 0.02000978, 0.00612013,
       0.00151717, 0.00949263, 0.00148975, 0.00183072, 0.00177311,
       0.00153454, 0.00912452, 0.00440076, 0.09190974, 0.00347175,
       0.00271852, 0.00162823, 0.00156344, 0.22189775, 0.00156289,
       0.00817172, 0.00157705, 0.00177236, 0.00174648, 0.0018643 ,
       0.00161106, 0.00169704, 0.00250301, 0.00280646, 0.00270028,
       0.00215105, 0.00168354, 0.00195193, 0.00201073, 0.00163011,
       0.00157523, 0.00156967, 0.00171114, 0.00143779, 0.00397507,
       0.17973685, 0.00198104, 0.00159236, 0.00204747, 0.00146683,
       0.00243851, 0.00854155, 0.00389146, 0.00179496, 0.00174829,
       0.00153278, 0.00160725, 0.00163806, 0.00168225, 0.00152878,
       0.00159134, 0.00187605, 0.00187412, 0.00147617, 0.00169

# Se genera el archivo para subir a Kaggle

In [20]:
test_set = pd.read_csv("test_set_xgb.csv")

In [21]:
test_set.columns

Index(['id', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'anio_publ'],
      dtype='object')

In [22]:
X_test_set = test_set.loc[:, test_set.columns != 'id']

## Armo el DF con las predicciones de test

In [23]:
XGB_ens_test_set = pd.DataFrame(index = test_set.index.copy())

In [24]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test_set)
    XGB_ens_test_set['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
XGB_ens_test_set.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,5.680446e+06,5359623.000,5.702577e+06,5449642.500,5.267412e+06,5927050.500,5667893.50,5.798230e+06,5587586.5,5632488.500,...,5.910642e+06,5175108.000,6.381258e+06,5.505993e+06,5.382352e+06,6.200663e+06,6.092716e+06,5.847866e+06,5748059.000,5437307.5
1,8.920947e+05,818902.375,7.676037e+05,690469.000,8.634014e+05,847262.375,847477.25,8.750496e+05,834616.0,837180.000,...,7.194673e+05,819181.875,9.211917e+05,8.570643e+05,8.992750e+05,8.277428e+05,8.490052e+05,8.218054e+05,917043.875,850014.0
2,2.387259e+06,2728783.000,2.559097e+06,2363850.750,2.583969e+06,2630720.750,2528766.25,2.368104e+06,2342233.5,2367905.500,...,2.519193e+06,2538569.500,2.456069e+06,2.482059e+06,2.582810e+06,2.553146e+06,2.440758e+06,2.353449e+06,2371260.250,2518026.5
3,1.221003e+06,1312173.875,1.222480e+06,1292531.375,1.099038e+06,1402850.375,1354771.00,1.031275e+06,1192381.5,1233050.375,...,1.040241e+06,1087237.750,1.163374e+06,1.294424e+06,1.352105e+06,1.172691e+06,1.439120e+06,1.291150e+06,1283227.375,1578799.0
4,5.644230e+05,616961.875,6.175370e+05,541452.875,5.484961e+05,532635.250,554821.50,6.035942e+05,583229.0,544496.000,...,6.139132e+05,603756.500,6.677337e+05,5.580501e+05,6.170163e+05,5.785553e+05,6.483061e+05,5.975401e+05,565824.625,570852.5


In [25]:
pred_a_kaggle = model_XGB.predict(XGB_ens_test_set)

In [26]:
df_a_kaggle = pd.DataFrame()
df_a_kaggle['id'] = test_set['id']
df_a_kaggle['target'] = pred_a_kaggle
df_a_kaggle.head()

,id,target
0,4941,6.246210e+06
1,51775,9.354144e+05
2,115253,2.455612e+06
3,299321,1.144174e+06
4,173570,5.854365e+05


In [27]:
df_a_kaggle.to_csv("subir_a_kaggle", header = True, index = False)